In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

import sys
sys.path.append("..")

from shared.utils.Website import Website

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
link_filter_system_prompt = """
You are the decider who are able to decide which links that are provided is relevant about the company. 
The links given are from a webpage that has been scraped, some may not be relevant, some are relevant.
Your job is to filter which links are relevant. I will give an example of the answer that you must strictly follow:
{
    "links": [
        {"type": "something page", "url": "https://example.com/to/do"},
        {"type": "about page", "url": "https://another.example.com/about"}
    ]
}
"""

In [4]:
print(link_filter_system_prompt)


You are the decider who are able to decide which links that are provided is relevant about the company. 
The links given are from a webpage that has been scraped, some may not be relevant, some are relevant.
Your job is to filter which links are relevant. I will give an example of the answer that you must strictly follow:
{
    "links": [
        {"type": "something page", "url": "https://example.com/to/do"},
        {"type": "about page", "url": "https://another.example.com/about"}
    ]
}



In [5]:
def link_filter_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [6]:
def filter_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_filter_system_prompt},
            {"role": "user", "content": link_filter_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [7]:
royal_url = "https://royalfx.co.id"
web = Website(royal_url)

In [8]:
filtered_links = filter_links(royal_url)

In [9]:
filtered_links

{'links': [{'type': 'home page', 'url': 'https://royalfx.co.id/'},
  {'type': 'company profile page',
   'url': 'https://royalfx.co.id/index.php/profil-perusahaan/'},
  {'type': 'legal page', 'url': 'https://royalfx.co.id/index.php/legalitas/'},
  {'type': 'forex page', 'url': 'https://royalfx.co.id/index.php/forex/'},
  {'type': 'loco london page',
   'url': 'https://royalfx.co.id/index.php/loco-london/'},
  {'type': 'crude oil page',
   'url': 'https://royalfx.co.id/index.php/crude-oil/'},
  {'type': 'platform page',
   'url': 'https://royalfx.co.id/index.php/platform/'},
  {'type': 'education page', 'url': 'https://royalfx.co.id/edukasi/'},
  {'type': 'forex description page',
   'url': 'https://royalfx.co.id/index.php/forex-description/'},
  {'type': 'gold description page',
   'url': 'https://royalfx.co.id/index.php/gold-description/'},
  {'type': 'oil description page',
   'url': 'https://royalfx.co.id/oil-description/'},
  {'type': 'news page', 'url': 'https://news.royalfx.co.id

In [10]:
def get_website_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = filter_links(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
analyzer_system_prompt = """
You are an assistant that helps analyze the contents of many relevant pages from a company website.
From your analysis, create a short yet informative summary in some form of brochure about the company for
giving an explanation about the company. Respond in markdown (not code blocks). Include details of company culture, customers and careers/jobs if you have
the information.
"""

In [17]:
def analyzer_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_website_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [18]:
def analyze_website(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": analyzer_system_prompt},
            {"role": "user", "content": analyzer_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    return result

In [19]:
result = analyze_website("Royal Trust Futures", "https://royalfx.co.id")

In [21]:
display(Markdown(result))

# Royal Trust Futures Brochure

## Welcome to Royal Trust Futures (ROYALFX)

### Your Trusted Trading Partner
At Royal Trust Futures, we pride ourselves on being a licensed official broker recognized by BAPPEBTI. We strive to provide a safe trading environment while empowering our traders with the tools and resources they need to succeed in the financial markets.

### Company Profile
- **Name:** PT. Royal Trust Futures
- **Industry:** Financial Trading
- **License:** Registered with BAPPEBTI
- **Contact:** [support@royalfx.co.id](mailto:support@royalfx.co.id) | +62 21 30304129

### Why Choose Us?
- **No Requotes**: Experience seamless trading with our advanced technology, ensuring quick market execution without requotes or price delays.
- **24/7 Customer Support**: Our dedicated customer service team is available around the clock via Live Chat to assist you with your trading needs.
- **Segregated Accounts**: Your funds are securely held in separate accounts, independently monitored by PT. Kliring Berjangka Indonesia.
- **Educational Resources**: We offer daily market analysis, webinars, tutorials, and e-books to enhance your trading education and skills.

### Our Offerings
#### Trading Products:
1. **Forex**: Trade currency pairs to capitalize on price fluctuations.
2. **Loco London**: Invest in one of the most stable commodities, ideal for portfolio diversification and protecting wealth.
3. **Crude Oil**: Engage in trading one of the world's most sought-after commodities, benefiting from dynamic price movements.

#### User-Friendly Platform:
Utilize our international platform, MetaTrader 4, compatible with multiple devices, providing quick market execution and a user-friendly interface.

### Career Opportunities
At Royal Trust Futures, we foster a dynamic and inclusive company culture. We welcome enthusiastic individuals eager to grow in the financial sector. Join us to embark on a rewarding career path where your contribution is valued and recognized.

### Join Us Today!
Experience the ease of accessing global financial markets with Royal Trust Futures. Sign up now to take the first step towards achieving your trading goals!

#### Get Started:
- **Open a Live Account**
- **Try Our Free Demo Account**
- **Explore Educational Resources**

For more information, please contact us at [support@royalfx.co.id](mailto:support@royalfx.co.id) or visit our website at [royalfx.co.id](http://royalfx.co.id).

---

*Trading involves risk and may not be suitable for all investors. Please ensure you understand the risks before you engage in trading.*